## Ex 10 Weekly dataset

**10.a) numerical and graphical summary of the Weekly data**

In [3]:
require(ISLR)
#SP500 weekly returns between 1990 and 2010
str(Weekly)
dim(Weekly)
summary(Weekly)

ERROR: Error in unlist(str_split(x, "\n"), recursive = FALSE, use.names = FALSE): lazy-load database '/Users/zimine/anaconda3/lib/R/library/stringr/R/stringr.rdb' is corrupt


In [ ]:
pairs(Weekly, cex=0.1, col=Weekly$Direction)
# we do not observe sppcifice relations other thena between volum and year
# similar ot the Smarket dataset

In [ ]:
## correlations appear to be low except Year~Volume
print(cor(Weekly[,-9]))

In [ ]:
plot(Volume~Year, data=Weekly,cex=0.3,main="volume versus year for weekly returns")
# we observe  volume increase up to 2008 then some decrease in 2009,2010

**10.b) Logistic regression**

In [ ]:
##logistic regression
glm.fit <- glm(Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume,data=Weekly, family=binomial)
print(summary(glm.fit))
# as we see we have a significant p-value for Lag2

**10.c) compute the confusion matrix  and overral error rate for logistic regression**

In [ ]:
logr.proba <- predict(glm.fit, type="response")
logr.pred <- rep("Down", length(logr.proba))
logr.pred[logr.proba > .5] <- "Up"

logr.cfm <- table(logr.pred, Weekly$Direction)
print(logr.cfm)

print(paste("log reg model correct rate:",mean(logr.pred == Weekly$Direction)))
print(paste("log reg model error rate:", 1 - mean(logr.pred == Weekly$Direction)))

## weeks that market is predicted to go  up is  557/(557+48) = 92.1%
## weeks that market is predicted to go down is 54 / (430 + 54) = 11.2 %

** optional,  check the predictive power of this model on test set, trained on training set 1990-2008**

In [ ]:
#split to training and test  sets
train_idx <- Weekly$Year < 2009
Weekly_train <- Weekly[train_idx,]
Weekly_test  <- Weekly[!train_idx,]
print(dim(Weekly_train))
print(dim(Weekly_test))

In [ ]:
##use the same model applied on training data set
glm.fit_tr <- glm(Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume
	          ,data=Weekly_train, family=binomial)

In [ ]:
#compute predictions on test dataset and print the confusion matrix
logr.proba_ts <- predict(glm.fit_tr, Weekly_test, type="response")
logr.pred_ts <- rep("Down", length(logr.proba_ts))
logr.pred_ts[logr.proba_ts > .5] <- "Up"
print(length(logr.pred_ts))

logr.cfm_ts <- table(logr.pred_ts, Weekly_test$Direction)
print(logr.cfm_ts)
print(paste("log reg same model correct rate on test:",mean(logr.pred_ts == Weekly_test$Direction)))
print(paste("log reg same model error rate on test:", 1 - mean(logr.pred_ts == Weekly_test$Direction)))

## as we can observe our eror rate on test data increases from 0.43  to 0.53!!!

** 10.d)  fit a logistic regression using only Lag2  using a training set and make predicions on test set**

In [ ]:
glm.fit_tr2 <- glm(Direction~Lag2,data=Weekly_train, family=binomial)
print(summary(glm.fit_tr2))

In [ ]:
#compute predictions on test dataset and print the confusion matrix
logr.proba_ts <- predict(glm.fit_tr2, Weekly_test, type="response")
logr.pred_ts <- rep("Down", length(logr.proba_ts))
logr.pred_ts[logr.proba_ts > .5] <- "Up"


logr.cfm_ts <- table(logr.pred_ts, Weekly_test$Direction)
print(logr.cfm_ts)
print(paste("log reg Lag2 model correct rate on test:",mean(logr.pred_ts == Weekly_test$Direction)))
print(paste("log reg Lag2 model error rate on test:", 1 - mean(logr.pred_ts == Weekly_test$Direction)))

## as we see Logistic regression using just Lag2 is much better compared to using all Lags 

**10.e) repead d) using LDA **

In [ ]:
##10.e)  repeat d) using LDA
require(MASS)
lda.fit <- lda(Direction~Lag2, data=Weekly, subset=train_idx)
lda.pred <- predict(lda.fit, Weekly_test)
lda.class <- lda.pred$class 

lda.cfm <- table(lda.class, Weekly_test$Direction)
print(lda.cfm)
print(paste("LDA Lag2 model correct rate on test:",mean(lda.class == Weekly_test$Direction)))
print(paste("LDA Lag2 model error rate on test:", 1 - mean(lda.class == Weekly_test$Direction)))


**10.f) repead d) using QDA **

In [ ]:
##10.f  repead d) using QDA
qda.fit <- qda(Direction~Lag2, data=Weekly, subset=train_idx)
qda.pred <- predict(qda.fit, Weekly_test)
qda.class <-qda.pred$class 
qda.cfm <- table(qda.class, Weekly_test$Direction)
print(qda.cfm)
print(paste("QDA Lag2 model correct rate on test:",mean(qda.class == Weekly_test$Direction)))
print(paste("QDA Lag2 model error rate on test:", 1 - mean(qda.class == Weekly_test$Direction)))

**10.f) repead d) using knnn with k=1 **

In [ ]:
##10.g  repeat d) knn with k=1
library(class)

train.X <- cbind(Weekly_train$Lag2)
test.X <- cbind(Weekly_test$Lag2)

#define a knn function to be ran later with different k
getKnnPrediction <- function(train, test, train_y, test_y, k=1){ 
	set.seed (1)
	knn.pred <- knn(train,test,train_y, k=k)
	cfm_knn <- table(knn.pred, test_y)
	print(cfm_knn) 
    print(paste("knn model correct rate on test for k =",k,":",
    	mean(knn.pred == test_y)))
    print(paste("knn model error rate on test for k =",k,":",
    	1-mean(knn.pred == test_y)))

}

getKnnPrediction(train=train.X, test=test.X,
	             train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=1)

**10.h) which of the methods above gives better results? **

The best error rate is for Lag2 as a sole predictor model is given by LDA and Logistic Regression

**10.i) experiment with different combination of predictors **

In [ ]:
## experimenting with Lag2 model knn using different k 
getKnnPrediction(train=train.X, test=test.X,
                 train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=1)
getKnnPrediction(train=train.X, test=test.X,
                 train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=2)

getKnnPrediction(train=train.X, test=test.X,
                 train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=3)

getKnnPrediction(train=train.X, test=test.X,
                 train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=4)

getKnnPrediction(train=train.X, test=test.X,
                 train_y=Weekly_train$Direction, test_y=Weekly_test$Direction,k=5)

#we see that the best error rate is given by k=4

In [ ]:
## use 1/2(Lag1+Lag2) as predictor
##combine lag1 and lag2 into averaged lag 
Weekly_t <- Weekly 
Weekly_t$Lag12 <- 0.5*(Weekly_t$Lag1+Weekly_t$Lag2)
Weekly_t_train <- Weekly_t[train_idx,]
Weekly_t_test  <- Weekly_t[!train_idx,]

lda.fit <- lda(Direction~Lag12, data=Weekly_t, subset=train_idx)
lda.pred <- predict(lda.fit, Weekly_t_test)
lda.class <- lda.pred$class 

lda.cfm <- table(lda.class, Weekly_t_test$Direction)
print(lda.cfm)
print(paste("LDA Lag12 model correct rate on test:",mean(lda.class == Weekly_t_test$Direction)))
print(paste("LDA Lag12 model error rate on test:", 1 - mean(lda.class == Weekly_t_test$Direction)))

#we observe that this approach makes a worse result than LDA using just L2
# knn with k=4 gives similar results as LDA on L2

In [ ]:
#check Log Reg with Lag1:lAg2   i.e. with inrecaction term

glm.fit_tr2 <- glm(Direction~Lag2:Lag1,data=Weekly_train, family=binomial)

logr.proba_ts <- predict(glm.fit_tr2, Weekly_test, type="response")
logr.pred_ts <- rep("Down", length(logr.proba_ts))
logr.pred_ts[logr.proba_ts > .5] <- "Up"


logr.cfm_ts <- table(logr.pred_ts, Weekly_test$Direction)
print(logr.cfm_ts)
print(paste("log reg Lag2:Lag1 model correct rate on test:",mean(logr.pred_ts == Weekly_test$Direction)))
print(paste("log reg Lag2:Lag1 model error rate on test:", 1 - mean(logr.pred_ts == Weekly_test$Direction)))



**we observe that the original model on Lag2  LogReg and LDA  plus knn with k=4 yielded the best results**